In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd

import xgboost as xgb
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization

import warnings
warnings.simplefilter("ignore")



tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)


file_list = glob.glob('../data/raw/training_set/log_0*.csv')
log_df = dd.read_csv(file_list)
# log_df = pd.read_csv('../data/raw/training_set/log_0_20180715_000000000000.csv')


In [2]:
def get_sim(df_hist, df_lookup, sim_file_list, score_name_list):
    df_hist['ListenYes'] = (df_hist['skip_2'] == False)*1
    df_hist['ListenYes'].replace(0, -1, inplace = True)
    df_hist = df_hist.groupby(['session_id', 'clus']).agg({'ListenYes':['sum']})
    df_hist = df_hist.reset_index()
    df_hist.columns = df_hist.columns.droplevel(level = 1) # take out the unwanted level
    df_pivot = pd.pivot_table(df_hist, values = 'ListenYes',index='session_id', columns='clus')
    df_pivot = df_pivot.fillna(0)
    
    
    for sim_file, score_name in zip(sim_file_list, score_name_list):
        sim_matrix = pd.read_csv(sim_file).drop(columns=['Unnamed: 0'])
        sim_matrix.columns = list(map(str, range(0,len(sim_matrix))))
        df_sim_session = df_pivot.dot(sim_matrix)/sim_matrix.sum()
        
        df_lookup[score_name] = df_sim_session.lookup(df_lookup['session_id'],df_lookup['clus'].astype(str))
    
    return df_lookup

In [ ]:
kmean100_df = pd.read_csv('../data/interim/all_data/mbKMeans100clusters.csv', usecols=['track_id','clus'])
kmean100_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)
log_df = log_df[['session_id','track_id_clean','skip_2','session_position','session_length']].merge(kmean100_df)

log_df_1 = log_df.loc[log_df['session_position']<(log_df['session_length']/2)].compute()

# as the entire dataset will be too big to train, 
# train the first 2 tracks of the 2nd half (prediction set) should be enough, 
# as these 2 contribute most to the spotify metric

half_cut = log_df['session_length']/2

log_df_2_1 = log_df.loc[(log_df['session_position']>=half_cut) & (log_df['session_position']<half_cut+1)]
log_df_2_1['weight'] = 2
log_df_2_2 = log_df.loc[(log_df['session_position']>=half_cut+1) & (log_df['session_position']<half_cut+2)]
log_df_2_2['weight'] = 1
                                       
# log_df_2 = pd.concat([log_df_2_1, log_df_2_2])
log_df_2 = dd.concat([log_df_2_1, log_df_2_2]).compute()


sim_file_list = ['../models/SVD/similarity/k100_CanbDist.csv',
                 '../models/SVD/similarity/k100_CosSim.csv',
                 '../models/SVD/similarity/k100_LinCorr.csv',
                 '../models/SVD/similarity/k100_ManhDist.csv',
                 '../models/SVD/similarity/k100_HammDist.csv',
                 '../models/SVD/similarity/k100_SpearCorr.csv',
                 '../models/SVD/similarity/k100_KendCorr.csv']
score_name_list = ['CanbDist100', 'CosSim100','LinCorr100','ManhDist100','HammDist100','SpearCorr100','KendCorr100']

df_lookup = get_sim(log_df_1, log_df_2, sim_file_list, score_name_list)

df_lookup = df_lookup.merge(tf_df)
df_lookup = pd.get_dummies(df_lookup, columns=['key','time_signature','mode'])

dtrain = xgb.DMatrix(df_lookup.drop(columns = ['session_id','track_id_clean','skip_2']), 
                     label=df_lookup['skip_2'],
                     weight = df_lookup['weight'])

In [16]:
params = {'max_depth': 3,
              'gamma': 0.5,
              'learning_rate':0.01,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'error'}
cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)

In [26]:
cv_result['test-error-mean'].iloc[-1]


0.33189355031779444

In [29]:
def bo_tune_xgb(max_depth, gamma ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'error'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=10, nfold=5)
    #Return the negative RMSE
    return 1-cv_result['test-error-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate':(0,0.01)
                                            })
xgb_bo.maximize(n_iter=5, init_points=8, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth |
-------------------------------------------------------------
| 1         | 0.6632    | 0.9008    | 0.0002473 | 4.137     |
| 2         | 0.6632    | 0.2555    | 0.002755  | 4.43      |
| 3         | 0.6644    | 0.05842   | 0.007041  | 5.593     |
| 4         | 0.667     | 0.3743    | 0.005592  | 8.79      |
| 5         | 0.6664    | 0.7489    | 0.002916  | 6.849     |
| 6         | 0.6666    | 0.09849   | 0.002702  | 7.371     |
| 7         | 0.6664    | 0.1743    | 0.005325  | 6.672     |
| 8         | 0.667     | 0.8666    | 0.003208  | 9.669     |
| 9         | 0.6664    | 0.0       | 0.01      | 10.0      |
| 10        | 0.667     | 1.0       | 0.005006  | 8.81      |
| 11        | 0.6669    | 0.7101    | 0.001927  | 9.15      |
| 12        | 0.6669    | 0.9838    | 0.009493  | 9.987     |
| 13        | 0.667     | 0.59      | 0.009418  | 8.349     |


In [36]:
xgb_bo.max['params']

{'gamma': 0.5899964093512567,
 'learning_rate': 0.009417981929522201,
 'max_depth': 8.34871018871217}

In [38]:
params = {'max_depth': 8,
              'gamma': 0.5899964093512567,
              'learning_rate':0.009417981929522201,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'error'}
cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)


In [39]:
cv_result

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.330065,0.000556,0.336290,0.001781
1,0.327947,0.000455,0.335058,0.001734
2,0.327521,0.000556,0.334215,0.001634
3,0.327208,0.000382,0.334010,0.001586
4,0.326928,0.000401,0.333675,0.001769
...,...,...,...,...
65,0.324187,0.000580,0.332452,0.001705
66,0.324150,0.000591,0.332451,0.001782
67,0.324103,0.000569,0.332460,0.001714
68,0.324076,0.000612,0.332460,0.001717
